In [ ]:
import re
char = re.compile('[^ 0-9a-zA-Zㄱ-ㅣ가-힣!?#.]')
insta_data = [] 
for url in url_list:
    try:
        driver.get(url)
        insta_id = driver.find_element_by_class_name('e1e1d').text
        hash_text_area = driver.find_element_by_class_name('EtaWk')
        hash_text = hash_text_area.text
        hash_text = char.sub('', hash_text)
        heart = ""
        try:
            heart = driver.find_element_by_class_name('Nm9Fw').text
            heart = heart.split(" ")[-1][:-1].replace(",","")
        except:
            heart = "0"
        date = driver.find_element_by_class_name('k_Q0X.NnvRN').text
        published_date = ""
        if "시간 전" in date:
            import datetime
            now = datetime.datetime.now()
            hours = date.split("시간 전")[0]
            published_date = now - datetime.timedelta(hours=int(hours))
            published_date = published_date.strftime('%m월 %d일'.encode('unicode-escape').decode()).encode().decode('unicode-escape')

        elif "일 전" in date:
            import datetime
            now = datetime.datetime.now()
            days = date.split("일 전")[0]
            published_date = now - datetime.timedelta(days=int(days))
            published_date = published_date.strftime('%m월 %d일'.encode('unicode-escape').decode()).encode().decode('unicode-escape')
        else:
            published_date = date
        data = {"Insta ID" : insta_id, "Heart": heart, "Date": published_date, "Content":hash_text}
        insta_data.append(data)
    except Exception as e:
        print(e)
        
print(insta_data)

In [20]:
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import xlsxwriter
from datetime import datetime
import requests
from io import BytesIO

today = datetime.today().strftime('%Y-%m-%d')
file_name = 'gmarket.xlsx'
workbook = xlsxwriter.Workbook(file_name)
ws = workbook.add_worksheet(today)
ws.write(0,0,'상품 이미지')
ws.write(0,1,'상품 명')
ws.write(0,2,'상품 원가')
ws.write(0,3,'상품 할인가')
ws.write(0,4,'상품 할인율')
ws.set_column(0,0, 50)
ws.set_column(1,1, 50)
ws.set_default_row(100)
count = 1
driver = webdriver.Chrome('chromedriver.exe')
driver.get('http://corners.gmarket.co.kr/Bestsellers')
wait = WebDriverWait(driver, 10)
for i in range(30):
    time.sleep(0.1)
    driver.find_element_by_css_selector('body').send_keys(Keys.PAGE_DOWN)

wait_state = wait.until(EC.presence_of_all_elements_located((By.TAG_NAME,'img')))
best_list = driver.find_elements_by_css_selector('div.best-list')[1]
best_list = best_list.find_elements_by_css_selector('ul > li')
for item in best_list:
    thumb = item.find_element_by_class_name('thumb')
    thumb_img = thumb.find_element_by_css_selector('img.lazy').get_attribute('src')
    item_name = item.find_element_by_class_name('itemname').text
    item_price = item.find_element_by_class_name('item_price')
    o_price = item_price.find_element_by_class_name('o-price').text
    s_price = item_price.find_element_by_class_name('s-price').text
    try:
        sale = s_price.split(" ")[1]
    except:
        sale = '0%'
    s_price = s_price.split(' ')[0]
    if o_price == '':
        o_price = s_price
    
    ws.write(count, 1, item_name)
    ws.write(count, 2, o_price)
    ws.write(count, 3, s_price)
    ws.write(count, 4, sale)
    
    try:
        if not 'no_image.gif' in thumb_img:
            res = requests.get(thumb_img)
            image_data = BytesIO(res.content)
            image_size = len(image_data.getvalue())
            if image_size > 0:
                cell = 'A'+str(count+1)
                ws.insert_image(count, 0, thumb_img, {'image_data':image_data, 'x_scale': 0.3, 'y_scale': 0.3})
    except:
        pass
    
    count += 1
workbook.close()
print('수집 종료')

수집 종료


In [13]:
import xlsxwriter
from datetime import datetime

today = datetime.today().strftime('%Y-%m-%d')
file_name = 'gmarket.xlsx'
workbook = xlsxwriter.Workbook(file_name)
ws = workbook.add_worksheet(today)
ws.write(0,0,'상품 이미지')
ws.write(0,1,'상품 명')
ws.write(0,2,'상품 원가')
ws.write(0,3,'상품 할인가')
ws.write(0,4,'상품 할인율')

workbook.close()

In [25]:
from selenium import webdriver
import time

keyword = input('검색어를 입력해주세요 : ')
driver = webdriver.Chrome('chromedriver.exe')
driver.get('https://search.naver.com/search.naver?where=news&sm=tab_jum&query=' + keyword)
now_page = driver.find_element_by_css_selector('div.sc_page > div.sc_page_inner > a ').get_attribute('href')
count = 0
url_list = []
while True:
    count += 1
    if count >= 5:
        break
    time.sleep(1)
    news_group = driver.find_element_by_css_selector('div.group_news > ul.list_news')
    news_list = news_group.find_elements_by_css_selector('li')
    for news in news_list:
        url = news.find_element_by_css_selector('a').get_attribute('href')
        url_list.append(url)

    next_page = driver.find_element_by_css_selector('div.sc_page > a.btn_next').get_attribute('href')
    '''
    if now_page == next_page:
        break
    '''
    now_page = next_page
    driver.get(now_page)
    
print(url_list)

검색어를 입력해주세요 : 경제
['https://search.naver.com/search.naver?where=news&sm=tab_jum&query=%EA%B2%BD%EC%A0%9C#', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=277&aid=0004936588', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=032&aid=0003085189', 'https://search.naver.com/search.naver?where=news&sm=tab_jum&query=%EA%B2%BD%EC%A0%9C#', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=104&oid=002&aid=0002199474', 'https://search.naver.com/search.naver?where=news&sm=tab_jum&query=%EA%B2%BD%EC%A0%9C#', 'https://search.naver.com/search.naver?where=news&sm=tab_jum&query=%EA%B2%BD%EC%A0%9C#', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=056&aid=0011081881', 'https://search.naver.com/search.naver?where=news&sm=tab_jum&query=%EA%B2%BD%EC%A0%9C#', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=001&aid=0012521964', 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=417&aid=0000714451', 'ht

In [26]:
url_list = list(set(url_list))


In [27]:
url_list

['https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=003&aid=0010602680',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=106&oid=609&aid=0000467255',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=003&aid=0010603500',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=421&aid=0005471895',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=076&aid=0003754270',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=001&aid=0012519990',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=119&aid=0002510726',
 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=003&aid=0010603502',
 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=%EA%B2%BD%EC%A0%9C&sort=0&photo=0&field=0&pd=0&ds=&de=&cluster_rank=23&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:all,a:all&start=11#',
 'https://news.naver.com/main/read.nhn?mod

In [35]:
from selenium import webdriver
import time
driver = webdriver.Chrome('chromedriver.exe')

def NaverNewsGetData(driver, url):
    driver.get(url)
    content = driver.find_element_by_class_name('content')
    article_info = content.find_element_by_class_name('article_info')
    article_title = article_info.find_element_by_id('articleTitle')
    title = article_title.text
    
    sponsor = article_info.find_element_by_class_name('sponsor')
    date = sponsor.find_element_by_class_name('t11').text
    
    article_body = content.find_element_by_class_name('article_body')
    article_content = article_body.find_element_by_id('articleBodyContents')
    
    content = article_content.text
    
    return title, date, content
def YnpNewsGetData(driver, url):
    browser.get(url)
    article_wrap = driver.find_element_by_id('articleWrap')
    title = article_wrap.find_element_by_class_name('tit-article').text
    date = article_wrap.find_element_by_css_selector('.article-sns-md > .share-info > .tt > em').text
    date = date.split(" ")[0]
    content = article_wrap.find_element_by_class_name('article').text   
    content = char.sub('', content) 
    return title, date, content
def HKNewsGetData(driver, url):
    browser.get(url)
    container = driver.find_element_by_id('container')
    article_top = container.find_element_by_class_name('article_top')
    title = article_top.find_element_by_class_name('title').text
    date_info = article_top.find_element_by_css_selector('.date_info > .date-published').text
    date = date_info.replace("입력", "")
    date = date.split(" ")[0]
    wrap_container = container.find_element_by_class_name('wrap_container')
    content = wrap_container.find_element_by_id('articletxt').text
    content = char.sub('', content) 
    return title, date, content
def DispatchGetData(driver, url):
    browser.get(url)
    column = driver.find_element_by_class_name('column.read')
    article_title = column.find_element_by_class_name('article-title')
    title = article_title.text
    date_info = column.find_element_by_class_name('article-created-at')
    date = date_info.text
    date = date.split(" ")[2]
    content = column.find_element_by_class_name('column.margin-top-helper.article-body.read').text
    content = char.sub('', content)
    return title, date, content


for url in url_list:
    if 'entertain.naver.com' in url:
        pass
    elif 'news.naver.com' in url:
        try:
            title, date, content = NaverNewsGetData(driver, url)
            print("-----------------------")
            print(title, date, content)
        except:
            pass
    elif 'app.yonhapnews.co.kr' in url:
        title, date, content = YnpNewsGetData(driver, url)
    elif 'news.hankyung.com' in url:
        title, date, content = HKNewsGetData(driver, url)
    elif 'dispatch.co.kr' in url:
        title, date, content= DispatchGetData(driver, url)
    

-----------------------
홍의락 대구시 경제부시장, 민주당 돌아가나? 2021.07.12. 오후 3:58 [대구=뉴시스] 홍의락 대구시 경제부시장이 대구시청기자실을 방문해 기자들의 질문에 대답하고 있다. 뉴시스DB. 2021.07.12. photo@newsis.com


[대구=뉴시스] 정창오 기자 = 홍의락 대구시 경제부시장이 대구시 입성 1년여 만에 공무원 신분에서 벗어날 것으로 보인다.

홍 부시장은 12일 오후 시청 기자실에서 가진 간담회에서 향후 정치적 행보와 관련해 “대구가 어려운 상황에서 봉사라고 생각해 부시장을 맡았다”면서 “이제는 마무리할 때가 됐다”고 말했다.

하지만 그는 부시장 사퇴 후 행보에 대해서는 “그다음 단계는 모르겠다”며 더불어민주당 복당 및 대구시장 선거 출마 여부에 대해서는 언급을 회피했다.

홍 부시장은 “시장과 이야기해봐야 하겠지만 8월쯤 되면 정리를 해야 하지 않느냐고 생각한다”며 사퇴가능성을 부각하면서도 “이것이 사의표명은 아니다”며 여지를 남겼다. 시청 주변에서는 사실상 사의를 표명한 것으로 해석하고 있다.

더불어민주당 출신으로 재선의원을 지낸 홍 부시장은 지난해 6월 말 권 시장의 요청에 의해 민주당을 탈당하고 대구시 경제부시장 직을 수락했다.

권 시장은 당시 페이스북에 “손을 맞잡고 코로나19 위기를 극복하고, 대구의 새로운 변화와 희망을 만들어 가겠다”며 환영했다.

민주당 대구시당도 논평을 내고 “대구시와 중앙정부, 여당과의 소통창구 이상의 역할을 충분히 해 낼 것”이라고 협치를 강조했다.

하지만 홍 부시장의 대구시 입성이 협치와 대구시 현안해결, 국비확보에 얼마나 성공적으로 기여했는지에 대한 평가는 엇갈리고 있다.
-----------------------
거시경제금융회의 2021.07.13. 오전 8:57 [서울=뉴시스] 이억원 기획재정부 차관이 13일 서울 명동 은행연합회에서 열린 거시경제금융회의에 참석해 발언하고 있다. (사진=기획재정부 제공) 2021.07.13. photo@ne

-----------------------
홍의락 대구시 경제부시장 사의 표명…"8월 말쯤 정리" 2021.07.12. 오후 3:12 사의 표명하는 홍의락 대구시 경제부시장
(대구=연합뉴스) 이덕기 기자 = 홍의락 대구시 경제부시장이 12일 기자간담회를 하고 사의를 표명하고 있다. 2021.7.12.
duck@yna.co.kr
(끝)


(대구=연합뉴스) 이덕기 기자 = 홍의락 대구시 경제부시장이 사의를 표명했다.

홍 부시장은 12일 기자간담회를 열어 향후 정치적 행보를 묻는 말에 "대구가 (지난해) 어려운 상황에서 봉사라고 생각했고 국회의원을 하면서 시민들과 한 이야기가 있으니 그 연장선상에서 부시장을 맡았다"면서 "이제는 마무리할 때가 됐다"고 말했다.

그러면서 "그다음 단계는 모르겠다"면서 향후 대구시장 선거 출마 여부에 대해서는 말을 아꼈다.

그는 "(권영진) 시장과 이야기해봐야 하겠지만 8월쯤 되면 정리를 해야 하지 않느냐고 생각한다"고 덧붙였다.

여당 국회의원 출신인 홍 부시장은 "(내년도) 예산도 (본인이) 직접 도와줄 부분이 있으면 정부안에 반영되도록 최대한 노력하겠다"고 했다.

지난 1년여간 경제부시장으로 있으면서 가장 큰 보람이 무엇이냐는 질문에 낙동강 취수원 이전 문제를 거론하면서 "당초 환경부는 낙동강 하류 부분만 처리하려고 했지만, 대구 취수원 이전 문제까지 넣도록 강력히 요구했다"고 설명했다.

duck@yna.co.kr
-----------------------
박승원 광명시장 "사회적경제로 광명시 변화 이끌것" 2021.07.13. 오후 1:10 박승원 광명시장은 3월 30일 시청 본관 1층에 마련된 사회적경제 전시 공간에서 사회적경제 홍보 영상을 촬영했다. / 사진제공=광명시
광명시(시장 박승원)는 사회적경제 활성화를 위한 조직개편으로 사회적경제과를 신설했다고 13일 밝혔다.

시에 따르면 기존 지역경제과 소속이었던 사회적경제팀과 창업지원과를 합쳐 사회적경제과를 신설하고 지역경제과는 기업지원과로 변경했다.

사회적경제과는

-----------------------
홍의락 대구시경제부시장 "8월 말쯤 정리할 것"…사의 표명 2021.07.12. 오후 3:32 대구시 경제부시장 직을 제안받은 홍의락 전 더불어민주당 의원이 지난해 6월25일 오후 대구 동구 신천동 더불어민주당 대구시당에서 열린 대구시당위원장 현안보고 및 간담회에 참석해 생각에 잠겨 있다. 2020.6.25/뉴스1 © News1 공정식 기자

(대구=뉴스1) 이재춘 기자 = 홍의락 대구시 경제부시장은 12일 "8월 말쯤 정리할 것"이라며 사실상 사의를 표명했다.

홍 부시장은 이날 시청에서 기자간담회를 갖고 "대구가 어려운 상황에서 봉사한다고 생각해 부시장을 맡았고, 이제는 마무리할 때가 됐다"며 이같이 말했다.

시장 출마와 관련해 그는 "(사퇴 이후) 다음 단계는 생각해 보지 않았다"며 구체적인 답변을 하지 않았다.

지역 정가에서는 홍 부시장이 대구시장에 출마할 것으로 보는 시각이 많다.

홍 부시장은 1년간의 활동에 대해 "취수원 이전과 엑스코선 연장 등이 기억에 남는다"고 했다.

더불어민주당 국회의원 출신인 홍 부시장은 지난해 7월1일 "대구형 협치의 성공 모델이 되기 위해서는 신뢰를 토대로 서로 노력하고 돕고 부족한 부분을 채워주는 줄탁동시(啐啄同時)여야 한다"며 대구시 경제부시장직을 수락, 임명됐다.
-----------------------
거시경제금융회의 2021.07.13. 오전 8:57 [서울=뉴시스] 이억원 기획재정부 차관이 13일 서울 명동 은행연합회에서 열린 거시경제금융회의에 참석해 발언하고 있다. (사진=기획재정부 제공) 2021.07.13. photo@newsis.com *재판매 및 DB 금지

[사진 영상 제보받습니다] 공감언론 뉴시스가 독자 여러분의 소중한 제보를 기다립니다. 뉴스 가치나 화제성이 있다고 판단되는 사진 또는 영상을 뉴시스 사진영상부(n-photo@newsis.com)로 보내주시면 적극 반영하겠습니다.
-----------------------
쿠바, 30년만에 반정부 